In [31]:
##Brute-Force, Exhaustive Search!
import pandas as pd
import numpy as np
#import scipy.stats
from sklearn.linear_model import LinearRegression  
import itertools
import copy
import warnings
import time
import pickle
from tabulate import tabulate

#loading strings here
#with open('stringlist.pkl', 'wb') as f:
       #pickle.dump(stringlist, f)
with open('stringlist.pkl', 'rb') as f:
      stringlist = pickle.load(f)

In [34]:
## Get test Data as a Pandas Dataframe! Make sure to have numerical data only! (pd.get_dummies might help)

file ="/Users/geoffreykasenbacher/Desktop/projects/branch and bound/Branch-and-Bound/Datasets/nuclear.csv"

data1 = pd.read_csv(file,sep=",").astype(float) #.drop(data.columns[1], axis=1)
data = data1.drop(data1.columns[0], axis=1)

Y = list(data)[4]
#X = list(data)[1:]

#print(data)

In [35]:
## exhaustive search - Brute Force. create class here

class BruteForce: 
    
    def __init__(self, data, Y, lowerbound, upperbound, adjusted_R2):
        self.data = data
        self.Y = Y
        self.lowerbound = lowerbound
        self.upperbound = upperbound
        self.adjusted_R2 = adjusted_R2
        
        datalist = list(self.data)
        self.X = [x for x in datalist if x != Y]
        
    def subset(self, X):
        _set = self.powerset(X)
        subset = []
        for i in _set:
            if self.upperbound >= len(i) >= self.lowerbound:
                subset.append(i)
        return subset
   
    def powerset(self, X):
        target = []
        def combinations(target,X):
            for i in range(len(X)):
                new_target = copy.copy(target)
                new_data = copy.copy(X)
                new_target.append(X[i])
                new_data = X[i+1:]
                pp.append(new_target)
                #print(new_target)
                combinations(new_target,new_data)
        pp = []
        combinations(target, X)
        return pp
    
    def regress(self, X, Y, data):
        regressor = LinearRegression()
        R = regressor.fit(data[X],data[Y])  
        
        #returns R^2
        r2 = regressor.score(data[X], data[Y])
        
        # Calculates adjusted R^2
        n = len(data)
        k = len(self.X)
        adj_r2 = ( 1 - (( (1-r2)*(n-1) )/ (n - k -1)) )
        
        if self.adjusted_R2 == True:
            return adj_r2
        elif self.adjusted_R2 == False:
            return r2
        else:
            raise ValueError("adjusted_r2 needs to be boolean: True or False (ノಠ益ಠ)ノ")
    
    def search(self):
        solutions = []
        #combos = self.powerset(self.X)
        combos = self.subset(self.X)
        for i in combos:
            solutions.append(self.regress(i, self.Y, self.data))  
        return solutions, combos
    
    def solve(self):
        sol, combos = self.search()
        highest_R2 = combos[np.argmax(sol)]
        R2 = max(sol)
        
        print("higehst R^2 is", R2, "with dependant variables:", highest_R2)
        return highest_R2, R2     

In [36]:
t, r2 = BruteForce(data, Y = 'date', lowerbound = 2, upperbound = 5, adjusted_R2 = True).solve()

t = (list(data))



higehst R^2 is 0.8194028439321454 with dependant variables: ['cost', 't1', 'ne', 'cum.n', 'pt']


In [37]:

## continue tomorrow



In [21]:
## this will solve all possible linear regressions and return the one with highest R^2

class ExhaustiveSearch: 
    
    def __init__(self, data, lowerbound, upperbound, adjusted_R2):
        self.data = data
        self.lowerbound = lowerbound
        self.upperbound = upperbound
        self.adjusted_R2 = adjusted_R2
        
        ## Raising Expections and Warnings here
        if len(list(self.data)) > 15:
            warnings.warn(stringlist[0], Warning)
        if str(type(self.data))  != "<class 'pandas.core.frame.DataFrame'>":
            raise TypeError(stringlist[1]) 
        if type(self.lowerbound) and type(self.upperbound) != int:
            raise TypeError(stringlist[2]) 
    
    def subset(self, X):
        _set = self.powerset(X)
        subset = []
        for i in _set:
            if self.upperbound >= len(i) >= self.lowerbound:
                subset.append(i)
        return subset
        
    def assign(self):
        XX = list(self.data)
        dic = {}
        for i in XX:
            Y = i
            #X = self.powerset([x for x in XX if x != Y])
            X = self.subset([x for x in XX if x != Y])
            dic[Y] = X
        return dic

    def powerset(self, X):
        target = []
        def combinations(target,X):
            for i in range(len(X)):
                new_target = copy.copy(target)
                new_data = copy.copy(X)
                new_target.append(X[i])
                new_data = X[i+1:]
                pp.append(new_target)
                combinations(new_target,new_data)
        pp = []
        combinations(target, X)
        return pp
    
    def regress(self, X, Y, data):
        regressor = LinearRegression()
        R = regressor.fit(self.data[X],self.data[Y])
        
        #returns R^2
        r2 = regressor.score(self.data[X], self.data[Y])
        
        #returns Adjusted R^2 
        n = len(self.data)
        k = len(list(self.data[X]))
        adj_r2 = (1 - ( ((1 - r2)*(n - 1))/(n - k - 1) ))
        
        if self.adjusted_R2 == True:
            return adj_r2
        elif self.adjusted_R2 == False:
            return r2
        else:
            raise ValueError(stringlist[3])
    
    def search(self):
        dic = self.assign()
        newdic = {}
        for y in dic:
            for x in dic[y]:
                newdic[self.regress(x, y, data)] = y, x
        new={}
        sd = sorted(newdic.items(), reverse=True)
        for k,v in sd:
            new[k] = v
        return new
    
    def output(self,results):
        headers = ['Best Models', 'Y (Dependant Variable)', ' X (Explanatoy Variables)', 'R^2']
        R_2 =[]
        for i in range(10):
            R_2.append(list(results.keys())[i])
            
        ##get variables from tuples
        content = []
        for index,i in enumerate(R_2):
            y, x = results[i]
            row = []
            row.append('# {:<10}'.format(1+index))
            row.append(y)
            row.append(x)
            row.append(i)
            content.append(row)

        table = tabulate(content, headers,tablefmt='orgtbl')
        return table

    def solve(self):
        results = self.search()
        table = self.output(results)
        
        highest_R2 = list(results.keys())[0]
        best_vars = results[highest_R2]
        
        yvar, xvar = best_vars
        
        #print("highest R^2 is", highest_R2, "with dependant/indepedant variables:", best_vars)
        #print('The best model found has {:^5}{} and {:^5}{} with R^2 of {}'.format('Y =',yvar,'X =',xvar, highest_R2))
        
        print(table)
        return best_vars, results

In [22]:
## ExhaustiveSearch returns a touple (y ,x) and the sorted set of all solutions as a dictionary {R^2:(y,x)}
wow = ExhaustiveSearch(data = data, lowerbound = 3, upperbound = 5, adjusted_R2 = True).solve()

| Best Models   | Y (Dependant Variable)   |  X (Explanatoy Variables)            |      R^2 |
|---------------+--------------------------+--------------------------------------+----------|
| # 1           | date                     | ['cost', 't1', 'ne', 'cum.n', 'pt']  | 0.854133 |
| # 2           | date                     | ['t1', 'ne', 'bw', 'cum.n', 'pt']    | 0.853815 |
| # 3           | date                     | ['cost', 't1', 'bw', 'cum.n', 'pt']  | 0.851942 |
| # 4           | date                     | ['t1', 'bw', 'cum.n', 'pt']          | 0.848934 |
| # 5           | date                     | ['cost', 't1', 'cap', 'ne', 'cum.n'] | 0.844442 |
| # 6           | date                     | ['t1', 'ct', 'bw', 'cum.n', 'pt']    | 0.844213 |
| # 7           | date                     | ['t1', 't2', 'bw', 'cum.n', 'pt']    | 0.843882 |
| # 8           | date                     | ['t1', 'pr', 'bw', 'cum.n', 'pt']    | 0.843693 |
| # 9           | date                     | ['t1'

In [23]:


## Next: add feature to exclude variables from Y combinations --> see variable Date!



In [24]:
## Add header acsii "caveman stats" for output function to stringlist and save!
stringlist = []
stringlist.append(c3po)
stringlist.append(dataerror)
stringlist.append(bounderror)
stringlist.append(valueerrorstring)
stringlist.append(yoda)

NameError: name 'c3po' is not defined

In [25]:
#print(wow)
headers = ['Best Models', 'Y (Dependant Variable)', ' X (Explanatoy Variables)', 'R^2']
R_2 =[]
for i in range(10):
    R_2.append(list(wow.keys())[i])
    
##get variables from tuples
content = []
for index,i in enumerate(R_2):
    y, x = wow[i]
    row = []
    row.append(str('# {:<10}'.format(1+index)))
    row.append(y)
    row.append(x)
    row.append(i)
    content.append(row)

table = tabulate(content, headers,tablefmt='orgtbl')
print(table)

AttributeError: 'tuple' object has no attribute 'keys'